# Linear Models

The simplest linear model for regression is called **linear regression** (called so because it is linear w.r.t. parameters), which is given by

$$
y(x, w) = w_0 + w_1x_1 + \dots + w_Dx_D
$$

where $x = (x_1, \dots, x_D)^T$. However, the expressivity of this model is very low, since iti s linear w.r.t. the input variables. This can be extended by considering **basis functions** $\phi_j(x)$ where

$$
y(x, w) = w_0 + \sum_{j=1}^{M-1}w_j\phi_j(x)
$$

The choice of basis function depends on the problem. For example, the "Gaussian" basis function can be given by

$$
\phi_j(x) = exp\{-\frac{(x - \mu_j)^2}{2s^2}\}
$$

or a sigmoid basis function given by 

$$
\phi_j(x) = \sigma(\frac{x - \mu_j}{s})
$$

where 

$$
\sigma(a) = \frac{1}{1 + exp(-a)}
$$

These can be seen in the plots below.

In [1]:
##TODO: Plot sigmoid and Gaussian and tanh basis functions

# Maximum Likelihood and Least Squares

Let $t$ be a target variable determined by a function $y(x, w)$ with additive Gaussian noise ($\epsilon ~ \mathcal{N}(0, \beta^{-1})$) such that

$$
t = y(x, w) + \epsilon
$$

then the probability of the target variable can be given by

$$
p(t|x, w, \beta) = \mathcal{N}(t|y(x, w), \beta^{-1})
$$

Thus the optimal solution will be the conditional mean of the target variable

$$
E[t|x] = \int tp(t|x)dt = y(x, w)
$$

Note that this implies that distribution of t given x is unimodal. Consider some i.i.d. data $X = \{x_1, \dots, x_N\}$ with corresponding $\{t_N\}$. Then the likelihood is given by

$$
p(t|X, w, \beta) = \prod_{n = 1}^N \mathcal{N}(t_n | w^T \phi(x_n), \beta^{-1})
$$

We wish to maximize the log likelihood to find the MLE for $w_{ML}$. First taking the log of the likelihood above gives

$$
\begin{align}
lnp(t|w, \beta) &= \sum_{n=1}^N ln(\mathcal{N}(t_n | w^T \phi(x_n), \beta^{-1}) \\
&= \sum_{n=1}^N ln(\frac{1}{(2\pi\beta^{-1})^{\frac{1}{2}}}) + \sum_{n=1}^N ln(e^{-\frac{1}{2\beta^{-1}}(t - w^T\phi(x_n))}) \\
&= \sum_{n=1}^N ln(1) - ln((2 \pi \beta^{-1})^{\frac{1}{2}}) - \sum_{n=1}^N \frac{1}{2 \beta^{-1}}(t - w^T \phi(x_n)) \\
&= \sum_{n=1}^N ln(1) - \sum_{n=1}^N ln((2 \pi \beta^{-1})^{\frac{1}{2}}) - \sum_{n=1}^N \frac{1}{2 \beta^{-1}}(t - w^T \phi(x_n))\\
&= \frac{N}{2}ln(\beta) - \frac{N}{2}ln(2\pi) + \beta E_D(w)
\end{align}
$$

where we defined $E_D(w)$ as the sum of squares error function 

$$
E_D(w) = \frac{1}{2} \sum_{n=1}^N(t_n - w^T\phi(x_n))^2
$$

Using the property $\frac{\partial}{A} (AB) = B^T$, we get the gradient

$$
\nabla ln p(t| w, \beta) = \beta \sum_{n=1}^N (t_n - w^T \phi(x_n)) \phi(x_n)^T
$$

and setting this equal to 0 and solving for $w$ gives the solution $w_{MLE} = (\Phi^T \Phi)^{-1}\Phi^T t$, known as the **normal equations** for least squares. Here $\Phi$ is the $N \times M$ **design matrix**. We can define the **Moore-Penrose pseudo-inverse** of the matrix as $\Phi^\dagger = (\Phi^T \Phi)^{-1}\Phi^T$. 

### Bias and Precision

We can repeat the calculation above on the bias $w_0$ and precision $\beta$ to get a better intuition of these parameters during regression as well. First consider the bias term. We can make the bias parameter explicit in the sum of squares error:

$$
E_D(w) = \frac{1}{2}\sum_{n=1}^N (t_n - w_0 - \sum_{j=1}^{M-1}w_j \phi_j(x_n))^2
$$

Then taking the derivative of the log likelihood above and setting equal to 0 gives:

$$
\begin{align}
\sum_{n=1}^N(t_n - w_0 - \sum_{j=1}^{M - 1} w_j \phi_j(x_n)) = 0 \\
\sum_{n=1}^N w_0 = \sum_{n=1}^N(t_n - \sum_{j=1}^{M - 1} w_j \phi_j(x_n)) \\
N w_0 = \sum_{n=1}^N(t_n - \sum_{j=1}^{M - 1} w_j \phi_j(x_n)) \\
w_0 = \bar{t} - \sum_{j=1}^{M - 1} w_j \bar{\phi_j}(x_n)
\end{align}
$$

where we define $\bar{t}$ and $\bar{\phi_j}$ as the average of the corresponding values. From this we see that the bias $w_0$ compensates for the difference between averages in the targets and the weighted sum of averages of the bias functions. 

Next we can consider the precision parameter $\beta$. Taking the derivative of the log likelihood w.r.t. precision and setting equal to 0 gives

$$
\begin{align}
\frac{N}{2\beta} - \frac{1}{2}\sum_{n=1}^N(t_n - w^T\phi(x_n))^2 = 0 \\
\frac{N}{2\beta} = \sum_{n=1}^N(t_n - w^T\phi(x_n))^2 \\
\frac{1}{\beta_{MLE}} = \frac{1}{N}\sum_{n=1}^N(t_n - w^T\phi(x_n))^2
\end{align}
$$

So the precision parameter is just the average error of the regression function's output and the target values, which makes sense given the parameter's name. 

## Sequential Learning

Note that we can rewrite least squares as a sequential learning problem by applying **stochastic gradient descent**. Let the iteration number be defined by $\tau$. Then we can define the gradient descent algorithm update on $w$ with:

$$
w^{(\tau + 1)} = w^{(\tau)} - \eta \nabla E_n
$$

where n denotes the nth data point, and $\eta$ is a learning rate. For the sum-of-squares error function, we get 

$$
w^{(\tau + 1)} = w^{\tau} + \eta(t_n - w^{(\tau)T}\phi_n)\phi_n
$$

where $\phi_n = \phi(x_n)$. This is known as **least-mean-squares**. 

## Regularized Least Squares

In general, the total error function that needs to be minimized is of the form

$$
E_D(w) + \lambda E_W(w)
$$

where $\lambda$ is the regularization coefficient. An example regularizer can be **weight decay** (or **parameter shrinkage** in statistics). 

$$
E_W(w) = \frac{1}{2}w^Tw
$$

This encourages weights to tend to 0 unless supported by the data. This error function keeps the error function quadractic in w, so it still has a closed form least-squares solution. 

More general, we can consider the p-norm

$$
E_W(w) = \frac{1}{2}\sum_{j=1}^M|w_j|^p
$$

Decreasing $p$ causes the model parameters $w_j$ to become more sparse. Thus, a model can be trained on limited data without overfitting by tuning the value of the regularization coeffcient $\lambda$ to make the model sparse enough.

## Multiple Target Outputs

Consider the case where we want to predict $K > 1$ target variables, which we denote with a vector t. We can then use the same basis functions to model all components of the target vector such that

$$
y(x, w) = W^T\phi(x)
$$

where $y \in \mathcal{R}^K$, $W \in \mathcal{R}^{M \times K}$, and $\phi(x) \in \mathcal{R}^M$, with the bias term $\phi_0(x) = 1$. We consider the conditional distribution of the target vector the be an isotropic Gaussian of form

$$
p(t|x, W, \beta) = \mathcal{N}(t|W^T\phi(x), \beta^{-1}I)
$$

then the log likelihood function is given by

$$
\begin{align}
lnp(T|X, W, \beta) &= \sum_{n=1}^N ln \mathcal{N}(t_n|W^T\phi(x_n),\beta^{-1}I) \\
&= \frac{NK}{2}ln(\frac{\beta}{2\pi}) - \frac{\beta}{2}\sum_{n=1}^N||t_n - W^T\phi(x_n)||^2
\end{align}
$$

where $T \in \mathcal{R}^{N \times K}$ is a matrix made from combining each $t_n^T$ as a row of T. Maximizing w.r.t. $W$ then gives

$$
W_{MLE} = (\Phi^T\Phi)^{-1}\Phi^T T
$$

Note that this means we only ever need to solve for the pseudo-inverse matrix $\Phi^{\dagger} = (\Phi^T\Phi)^{-1}\Phi^T$ and share it between all target values. This also extends to general non-isotopic Gaussian noise -- this derivation is found in the appendix.

# Appendix

*Statement: (PRML Exercise 3.1)* The $tanh$ function is related to the logistic sigmoid by 

$$
tanh(a) = 2\sigma(2a) - 1
$$

<details>
  <summary>Click to view proof</summary>

*Proof:* Remember that the sigmoid and tanh functions are given by

$$
\begin{align}
tanh(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}}\\
\sigma(a) = \frac{1}{1 + e^{-a}}
\end{align}
$$

Thus we see that 

$$
\begin{align}
2 \sigma(2a) - 1 &= \frac{2}{1 + e^{-2a}} - 1 \\
&= \frac{1 - e^{-2a}}{1 + e^{-2a}} \frac{e^a}{e^a} \\
&= \frac{e^a - e^{-a}}{e^a + e^{-a}}
\end{align}
$$

So we see that the equivalence holds. Now consider a general linear combination of tanh and sigmoid functions of the form:

$$
\begin{align}
y(x, u) = u_0 + \sum_{j=1}^M u_j tanh(\frac{x - \mu_j}{2s}) \\
y(x, w) = w_0 + \sum_{j=1}^M w_j \sigma(\frac{x - \mu_j}{s})
\end{align}
$$

where $u$ and $w$ are different parameters. Using the relation above we can express these parameters as a scalar transformation from one to the other. Consider the combination of tanh functions.

$$
\begin{align}
y(x, u) &= u_0 + \sum_{j=1}^M u_j tanh(\frac{x - \mu_j}{2s}) \\
&= u_0 + \sum_{j=1}^M u_j [2 \sigma(\frac{x - \mu_j}{s}) - 1] \\
\end{align}
$$

Comparing this to $y(x, w)$ we see

$$
\begin{align}
w_j \sigma &= u_j (2 \sigma - 1) \\
w_j = \frac{2 \sigma - 1}{\sigma} u_j
\end{align}
$$

so we have

$$
\begin{align}
w_j &= \frac{2, \sigma - 1}{\sigma} u_j,  \, \, \, 1 \leq j \leq M \\
w_0 &= u_0
\end{align}
$$
$\blacksquare$
</details>

# Appendix

*Statement: (PRML Exercise 3.1)* The $tanh$ function is related to the logistic sigmoid by 

$$
tanh(a) = 2\sigma(2a) - 1
$$

<details>
  <summary>Click to view proof</summary>
*Proof:* Remember that the sigmoid and tanh functions are given by

$$
\begin{align}
tanh(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}}\\
\sigma(a) = \frac{1}{1 + e^{-a}}
\end{align}
$$

Thus we see that 

$$
\begin{align}
2 \sigma(2a) - 1 &= \frac{2}{1 + e^{-2a}} - 1 \\
&= \frac{1 - e^{-2a}}{1 + e^{-2a}} \frac{e^a}{e^a} \\
&= \frac{e^a - e^{-a}}{e^a + e^{-a}}
\end{align}
$$

So we see that the equivalence holds. Now consider a general linear combination of tanh and sigmoid functions of the form:

$$
\begin{align}
y(x, u) = u_0 + \sum_{j=1}^M u_j tanh(\frac{x - \mu_j}{2s}) \\
y(x, w) = w_0 + \sum_{j=1}^M w_j \sigma(\frac{x - \mu_j}{s})
\end{align}
$$

where $u$ and $w$ are different parameters. Using the relation above we can express these parameters as a scalar transformation from one to the other. Consider the combination of tanh functions.

$$
\begin{align}
y(x, u) &= u_0 + \sum_{j=1}^M u_j tanh(\frac{x - \mu_j}{2s}) \\
&= u_0 + \sum_{j=1}^M u_j [2 \sigma(\frac{x - \mu_j}{s}) - 1] \\
\end{align}
$$

Comparing this to $y(x, w)$ we see

$$
\begin{align}
w_j \sigma &= u_j (2 \sigma - 1) \\
w_j = \frac{2 \sigma - 1}{\sigma} u_j
\end{align}
$$

so we have

$$
\begin{align}
w_j &= \frac{2, \sigma - 1}{\sigma} u_j,  \, \, \, 1 \leq j \leq M \\
w_0 &= u_0
\end{align}
$$
$\blacksquare$
</details>

*Statement: (PRML Exericise 1.25)* For a squared loss function of multiple target variables $t$, given by

$$
E[L(t, y(x)) = \int \int ||y(x) - t||^2 p(x, t)dxdt
$$

the function for which this loss is minimized is the conditional mean $y(x) = E_t[t|x]$. This reduces to the single target variable equation for a single target t.

*Proof:* **TODO**

*Statement: (PRML Exercise 3.2)* The matrix $\Phi(\Phi^T\Phi)^{-1}\Phi^T$ takes any vector v and projects it onto the space spanned by the columns of $\Phi$. 

*Proof:* **TODO**

*Statement: (PRML Exercise 3.5)* The minimization of the regularized error function 

$$
E_W(w) = \frac{1}{2}\sum_{j=1}^M|w_j|^p
$$

is equivalent to minimizing the unregularized sum-of-squares error

$$
E_D(w) = \frac{1}{2}\sum_{n=1}^N(t_n - w^T\phi(x_n))^2
$$

subject to the constraint

$$
\sum_{j=1}^M|w_j|^p \leq \eta
$$

*Proof:* **TODO**

*Statement: (PRML Exercise 3.6)* Consider a linear basis function regression model with a multivariate target variable t having a Gaussian distribution

$$
p(t|W, \Sigma) = \mathcal{N}(t| y(x, W), \Sigma)
$$

where $y(x, w) = W^T\phi(x)$. The maximum likelihood solution $W_{MLE}$ has the property that each column is given by an expression of the form 

$$
W_{MLE} = (\Phi^T\Phi)^{-1}\Phi^Tt
$$

which is the solution for an isotropic noise distribution. Also the maximum likelihood solution for the general covariance matrix $\Sigma$ is given by

$$
\Sigma = \frac{1}{N}\sum_{n=1}^N(t_n - W_{MLE}^T \phi(x_n))(t_n - W_{MLE}^T \phi(x_n))^T
$$

*Proof:* **TODO**

*Statement:* Based on the solution $w_{MLE}$ above, we see that if two or more basis vectors $\varphi_j$ (columns of the design matrix $\Phi$), are co-linear, and thus close to degenerate, the matrix $\Phi^T\Phi$ is close to singular. This means the computation $(\Phi^T\Phi)^{-1}$ is very expensive. According to Section 3.1.2, the addition of a regularization term ensures that $\Phi^T\Phi$ is non-singular, even in the presence of degeneracies.

*Proof:* **TODO**

**TODO:** include reference to Moore-Penrose pseudo-inverse. 